In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
%matplotlib inline
from scipy.integrate import quad
from scipy.special import gamma, gammainc, gammaln
from scipy.interpolate import interp1d

In [ ]:
true_a,true_r_s = 0.65463, 15.

In [ ]:
def ln_einasto(r, a, r_s):
    lnC1 = np.log(a) - np.log(r_s) - gammaln(1/a)
    return -(r/r_s)**a + lnC1

def ln_gaussian(x, mu, std):
    return -0.5*(x-mu)**2/std**2 - np.log(std) - 0.5*np.log(2*np.pi)

In [ ]:
r_grid = np.logspace(-1, 3., 1024)
cdf = np.array([quad(lambda *args: np.exp(ln_einasto(*args)), 0, rr, args=(true_a, true_r_s))[0] 
                for rr in r_grid])

In [ ]:
plt.semilogx(r_grid, ln_einasto(r_grid, true_a, true_r_s))

In [ ]:
cdf_func = interp1d(cdf, r_grid)

In [ ]:
# sample some true distances that follow the profile
n_data = 128
uu = np.random.uniform(cdf.min(), cdf.max(), size=n_data)
true_r = cdf_func(uu)

In [ ]:
d_err = true_r * 0.1 # 10% distance error
d = np.random.normal(true_r, d_err)

In [ ]:
bins = np.logspace(-1, 2.5, 18)
plt.hist(true_r, bins=bins, alpha=0.4)
plt.hist(d, bins=bins, alpha=0.4)
plt.xscale('log')

In [ ]:
def ln_integrand(r, a, r_s, d, d_err):
    return ln_einasto(r, a, r_s) + ln_gaussian(r, d, d_err)

def integrand(r, a, r_s, d, d_err):
    return np.exp(ln_integrand(r, a, r_s, d, d_err))

In [ ]:
# check normalizations
for i in range(8):
    _a = np.random.uniform(0.3, 0.9)
    _rs = np.random.uniform(5., 25.)
    _d = np.random.uniform(0, 250.)
    
    val,_ = quad(integrand, 0, np.inf, args=(_a, _rs, _d, 0.2*_d), epsabs=1E-13)
#     val,_ = quad(lambda *args: np.exp(ln_einasto(*args)), 0, np.inf, 
#                  args=(_a, _rs), epsabs=1E-13)
#     val,_ = quad(lambda *args: np.exp(ln_gaussian(*args)), 0, np.inf, 
#                  args=(_d, _d*0.2), epsabs=1E-13)
    
    print(val)

In [ ]:
def marg_ln_likelihood(p, ds, d_errs):
    a, r_s = p
    
    ln_l = 0.
    for d,d_err in zip(ds, d_errs):
        val,err = quad(integrand, 0, np.inf, args=(a, r_s, d, d_err), epsabs=1E-13)
        if np.abs(err/val) > 0.1:
            # print(d, d_err)
            # print(val, err)
            # raise ValueError("fail")
            return -np.inf
        ln_l += np.log(val)
    
    return ln_l

In [ ]:
quad(integrand, 0, np.inf, args=(val, true_r_s, 208.007536811, 18.0238195762), epsabs=1E-13)

In [ ]:
vals = np.linspace(0.5, 0.9, 128)
lls = np.zeros_like(vals)
for i,val in enumerate(vals):
    lls[i] = marg_ln_likelihood([val, true_r_s], d, d_err)

In [ ]:
fig,axes = plt.subplots(1, 2, figsize=(12,4))
axes[0].plot(vals[lls<0], lls[lls<0])
axes[1].plot(vals[lls<0], np.exp(lls[lls<0]-lls[lls<0].max()))